# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [3]:
import dask.dataframe as dd

/tmp/ipykernel_15916/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob
import dask.dataframe as dd

# Set the PRICE_DATA path
PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
dd_px = dd.read_parquet(parquet_files)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [21]:
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = np.concatenate([[np.nan], x['Close'].values[:-1]]) if 'Close' in x.columns else np.nan,
        Adj_Close_lag_1 = np.concatenate([[np.nan], x['Adj_Close'].values[:-1]]) if 'Adj_Close' in x.columns else np.nan,
        returns = x['Adj_Close'] / x['Adj_Close_lag_1'] - 1 if 'Adj_Close' in x.columns and 'Adj_Close_lag_1' in x.columns else np.nan,
        hi_lo_range = x['High'] - x['Low'] if 'High' in x.columns and 'Low' in x.columns else np.nan
    ).assign(
        positive_return = (x['returns'] > 0).astype(int) if 'returns' in x.columns else np.nan
    )
))

# Display the result
dd_feat

/tmp/ipykernel_15916/629566077.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,positive_return
npartitions=11217,,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [22]:
# Write your code below.
import pandas as pd

# Convert Dask DataFrame to Pandas DataFrame
pd_feat = dd_feat.compute()

# Adding rolling average return calculation
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(drop=True)

# Display the updated DataFrame
print(pd_feat.head())



             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
IP     2018-01-02  55.587120  56.666668  55.170456  56.600380  42.194801   
IP     2018-01-03  56.770832  57.604168  56.619316  57.348484  42.752502   
IP     2018-01-04  57.651516  58.598484  57.509472  58.314396  43.472569   
IP     2018-01-05  58.589016  58.607956  58.200760  58.428028  43.557285   
IP     2018-01-08  58.522728  59.128788  58.380684  58.835228  43.860832   

         Volume     sector                                       subsector  \
ticker                                                                       
IP      3355651  Materials  Paper & Plastic Packaging Products & Materials   
IP      2886893  Materials  Paper & Plastic Packaging Products & Materials   
IP      2705789  Materials  Paper & Plastic Packaging Products & Materials   
IP      1829098  Materials  Paper & Plastic Packaging Products & Materials   

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Some operations more efficiently handled in-memory with Pandas (such as rolling calculations for smaller datasets) may not benefit significantly from Dask parallelism. It was necessary given Dask's lazy execution that allows the chaining of operations without immediate computation but for specific operations like rolling(10).mean(), Pandas provide direct support, whereas Dask may require more complex handling due to its distributed nature.

+ Would it have been better to do it in Dask? Why?
If the dataset is massive, Dask excels in parallelizing operations across multiple cores or nodes, significantly enhancing performance and efficiency in data processing workflows. This capability is especially beneficial when dealing with datasets that exceed single-machine memory limits. Additionally, Dask's ability to scale and process efficiently across clusters optimizes computational resources, making it ideal for complex workflows that require distributed computing infrastructure and scalability beyond what a single machine can provide.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.